In [119]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

### data import, cleaning

In [120]:
df = pd.read_csv("D:\\Github\\Science\\NEO\\cneos_closeapproach_data.csv", sep=",")

In [121]:
df.head(2)

,Object,Close-Approach (CA) Date,CA Distance Nominal (LD | au),CA Distance Minimum (LD | au),V relative (km/s),V infinity (km/s),H (mag),Diameter,Unnamed: 8
0,(2020 VT4),2020-Nov-13 17:21 ± < 00:01,0.02 | 4.51e-5,0.02 | 4.50e-5,13.43,7.88,28.7,4.8 m - 11 m,bK20V04T
1,(2020 QG),2020-Aug-16 04:09 ± < 00:01,0.02 | 6.23e-5,0.02 | 6.22e-5,12.33,8.15,29.9,2.8 m - 6.2 m,bK20Q00G


In [122]:
df.dtypes

Object                            object
Close-Approach (CA) Date          object
CA Distance Nominal (LD | au)     object
CA Distance Minimum (LD | au)     object
V relative (km/s)                float64
V infinity (km/s)                float64
H (mag)                          float64
Diameter                          object
Unnamed: 8                        object
dtype: object

In [123]:
df["LD"] = df["CA Distance Nominal (LD | au)"].str.split(' ', expand = True)[0]

In [124]:
df.head(2)

,Object,Close-Approach (CA) Date,CA Distance Nominal (LD | au),CA Distance Minimum (LD | au),V relative (km/s),V infinity (km/s),H (mag),Diameter,Unnamed: 8,LD
0,(2020 VT4),2020-Nov-13 17:21 ± < 00:01,0.02 | 4.51e-5,0.02 | 4.50e-5,13.43,7.88,28.7,4.8 m - 11 m,bK20V04T,0.02
1,(2020 QG),2020-Aug-16 04:09 ± < 00:01,0.02 | 6.23e-5,0.02 | 6.22e-5,12.33,8.15,29.9,2.8 m - 6.2 m,bK20Q00G,0.02


In [125]:
df["LD"] = pd.to_numeric(df["LD"], errors='coerce')

In [126]:
df.dtypes

Object                            object
Close-Approach (CA) Date          object
CA Distance Nominal (LD | au)     object
CA Distance Minimum (LD | au)     object
V relative (km/s)                float64
V infinity (km/s)                float64
H (mag)                          float64
Diameter                          object
Unnamed: 8                        object
LD                               float64
dtype: object

In [127]:
df.index = np.arange(1, len(df) + 1 )
df["x_axis"] = df.index
df

,Object,Close-Approach (CA) Date,CA Distance Nominal (LD | au),CA Distance Minimum (LD | au),V relative (km/s),V infinity (km/s),H (mag),Diameter,Unnamed: 8,LD,x_axis
1,(2020 VT4),2020-Nov-13 17:21 ± < 00:01,0.02 | 4.51e-5,0.02 | 4.50e-5,13.43,7.88,28.7,4.8 m - 11 m,bK20V04T,0.02,1
2,(2020 QG),2020-Aug-16 04:09 ± < 00:01,0.02 | 6.23e-5,0.02 | 6.22e-5,12.33,8.15,29.9,2.8 m - 6.2 m,bK20Q00G,0.02,2
3,(2021 UA1),2021-Oct-25 03:07 ± < 00:01,0.02 | 6.30e-5,0.02 | 6.29e-5,15.84,12.89,31.8,1.1 m - 2.5 m,bK21U01A,0.02,3
4,(2011 CQ1),2011-Feb-04 19:39 ± < 00:01,0.03 | 7.92e-5,0.03 | 7.92e-5,9.69,5.17,32.1,1.0 m - 2.3 m,bK11C01Q,0.03,4
5,(2019 UN13),2019-Oct-31 14:45 ± < 00:01,0.03 | 8.43e-5,0.03 | 8.31e-5,12.85,10.09,32.0,1.1 m - 2.4 m,bK19U13N,0.03,5
...,...,...,...,...,...,...,...,...,...,...,...
30997,415713 (1998 XX2),1998-Nov-18 12:47 ± < 00:01,19.46 | 0.05000,19.46 | 0.05000,7.23,7.22,19.9,280 m - 620 m,a0415713,19.46,30997
30998,(2020 PX5),2118-May-17 05:18 ± 8_01:23,19.46 | 0.05000,16.75 | 0.04303,10.72,10.71,29.6,3.3 m - 7.3 m,bK20P05X,19.46,30998
30999,(2019 UN3),2176-Oct-20 12:12 ± 22:17,19.46 | 0.05000,19.21 | 0.04935,16.93,16.92,24.2,38 m - 86 m,bK19U03N,19.46,30999
31000,153814 (2001 WN5),2196-Jul-08 23:14 ± 2_19:34,19.46 | 0.05000,15.53 | 0.03991,8.78,8.78,18.3,0.932±0.011 km,a0153814,19.46,31000


### calculation

In [128]:
df["distance_km"] = df["LD"]*394_399

In [129]:
df["zero"]=0
df["earth"] = 6_371
df["ISS"] = 6_371+408
df["geostationary"] = 35_786

In [130]:
df = df.loc[df["LD"] < 0.2]

In [133]:
df.head()

,Object,Close-Approach (CA) Date,CA Distance Nominal (LD | au),CA Distance Minimum (LD | au),V relative (km/s),V infinity (km/s),H (mag),Diameter,Unnamed: 8,LD,x_axis,distance_km,zero,earth,ISS,geostationary
1,(2020 VT4),2020-Nov-13 17:21 ± < 00:01,0.02 | 4.51e-5,0.02 | 4.50e-5,13.43,7.88,28.7,4.8 m - 11 m,bK20V04T,0.02,1,7887.98,0,6371,6779,35786
2,(2020 QG),2020-Aug-16 04:09 ± < 00:01,0.02 | 6.23e-5,0.02 | 6.22e-5,12.33,8.15,29.9,2.8 m - 6.2 m,bK20Q00G,0.02,2,7887.98,0,6371,6779,35786
3,(2021 UA1),2021-Oct-25 03:07 ± < 00:01,0.02 | 6.30e-5,0.02 | 6.29e-5,15.84,12.89,31.8,1.1 m - 2.5 m,bK21U01A,0.02,3,7887.98,0,6371,6779,35786
4,(2011 CQ1),2011-Feb-04 19:39 ± < 00:01,0.03 | 7.92e-5,0.03 | 7.92e-5,9.69,5.17,32.1,1.0 m - 2.3 m,bK11C01Q,0.03,4,11831.97,0,6371,6779,35786
5,(2019 UN13),2019-Oct-31 14:45 ± < 00:01,0.03 | 8.43e-5,0.03 | 8.31e-5,12.85,10.09,32.0,1.1 m - 2.4 m,bK19U13N,0.03,5,11831.97,0,6371,6779,35786


### visualize

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots(figsize=(16, 9))
plt.plot(df["distance_km"],color="blue", alpha=0.7, marker=".", markersize=10,
         linestyle="", label="Near earth object")

# plt.plot(df["zero"],color="brown", alpha=0.7, marker="", markersize=10,
#          linestyle="-", label="")

# plt.plot(df["earth"],color="brown", alpha=0.7, marker="", markersize=10,
#          linestyle="-", label="")

plt.fill_between(df.x_axis, df['earth'], df['zero'], color='blue', alpha=0.5,
                 label=f'earth')


plt.plot(df["geostationary"],color="red", alpha=0.7, marker="", markersize=10,
         linestyle="-", label="geostationary orbit")

plt.plot(df["ISS"],color="orange", alpha=0.7, marker="", markersize=10,
         linestyle="-", label="ISS orbit")



ax.set_xlabel("NEO No.", fontsize=25)
ax.set_ylabel("Distance [km]", color="blue", fontsize=25)

ax.tick_params(labelsize=25)

plt.legend(loc='center',
           bbox_to_anchor=(0.5, -0.2),
           fancybox=True,
           shadow=True,
           ncol=3,
           fontsize=20)

plt.title(f' Near earth objects (Data: NASA) \n\n', fontsize=25)
plt.suptitle(f'Filter: distance < 0.2 Lunar Distances \n https://cneos.jpl.nasa.gov/ca/ \n\n', fontsize=15, y=0.91)

plt.savefig("D:\\Github\\Science\\NEO\\NEO.png", dpi=300, bbox_inches='tight')

